**FASE 2: SOLUÇÃO DE BI & ANALYTICS (Censo Detran-SP Out/2025)**

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from google.colab import files
import io

In [2]:
uploaded = files.upload()

Saving condutores_habilitados_ativos.csv to condutores_habilitados_ativos.csv


In [6]:
try:
    # Lê direto do disco, já que o upload salvou o arquivo
    df = pd.read_csv('condutores_habilitados_ativos.csv', sep=',', encoding='latin1')
    print(f"Arquivo lido com sucesso! Linhas: {len(df)}")
except FileNotFoundError:
    print("ERRO: O arquivo não está na pasta. Rode a célula de upload novamente ou arraste o arquivo para a aba lateral.")

Arquivo lido com sucesso! Linhas: 165574


In [7]:
df.head()

,codigo_municipio,descricao_municipio,categoria_cnh,genero,exerce_atividade_remunerada,faixa_etaria,pessoa_com_deficiencia,condutor_bloqueado,qtd_condutores,mes_ref,ano_ref
0,7099,SAO JOSE DOS CAMPOS,B,MASCULINO,S,51-60 ANOS,N,N,1545,10,2025
1,6411,FERNANDOPOLIS,E,MASCULINO,S,61-70 ANOS,N,N,80,10,2025
2,6493,IBITINGA,AB,MASCULINO,N,61-70 ANOS,N,N,513,10,2025
3,7145,SOROCABA,AD,MASCULINO,S,51-60 ANOS,N,N,1773,10,2025
4,6313,CARAPICUIBA,B,MASCULINO,N,51-60 ANOS,N,N,7261,10,2025


In [8]:
df_clean = df[
    (df['categoria_cnh'].str.contains('B', na=False)) &
    (df['condutor_bloqueado'] == 'N')
].copy()

Converter a coluna de quantidade para número (se tiver erro vira 0)

In [9]:
df_clean['qtd_condutores'] = pd.to_numeric(df_clean['qtd_condutores'], errors='coerce').fillna(0)

Aplicar Filtros:
- Categoria TEM QUE CONTER 'B' (Pega B, AB, etc)
- Condutor NÃO pode estar bloqueado (N)

In [10]:
df_clean = df[
    (df['categoria_cnh'].str.contains('B', na=False)) &
    (df['condutor_bloqueado'] == 'N')
].copy()

print(f"Linhas após filtros: {len(df_clean)}")

Linhas após filtros: 54622


**CRIANDO OS KPIs**

Agrupar por Perfil (Cidade + Idade + Gênero + Status EAR)

In [11]:
df_grouped = df_clean.groupby(
    ['descricao_municipio', 'faixa_etaria', 'genero', 'exerce_atividade_remunerada']
)['qtd_condutores'].sum().reset_index()

In [13]:
df_grouped.head(2)

,descricao_municipio,faixa_etaria,genero,exerce_atividade_remunerada,qtd_condutores
0,ADAMANTINA,101-120 ANOS,MASCULINO,N,17
1,ADAMANTINA,18-21 ANOS,FEMININO,N,86


In [15]:
len(df_grouped)

23172

Pivotar (Transformar linhas em colunas para comparar S vs N)

In [16]:
df_pivot = df_grouped.pivot_table(
    index=['descricao_municipio', 'faixa_etaria', 'genero'],
    columns='exerce_atividade_remunerada',
    values='qtd_condutores',
    fill_value=0
).reset_index()

In [18]:
df_pivot.head()

exerce_atividade_remunerada,descricao_municipio,faixa_etaria,genero,N,S
0,ADAMANTINA,101-120 ANOS,MASCULINO,17.0,0.0
1,ADAMANTINA,18-21 ANOS,FEMININO,86.0,342.0
2,ADAMANTINA,18-21 ANOS,MASCULINO,55.0,443.0
3,ADAMANTINA,22-25 ANOS,FEMININO,296.0,288.0
4,ADAMANTINA,22-25 ANOS,MASCULINO,291.0,318.0


In [19]:
df_pivot.columns.name = None

In [20]:
df_pivot.head()

,descricao_municipio,faixa_etaria,genero,N,S
0,ADAMANTINA,101-120 ANOS,MASCULINO,17.0,0.0
1,ADAMANTINA,18-21 ANOS,FEMININO,86.0,342.0
2,ADAMANTINA,18-21 ANOS,MASCULINO,55.0,443.0
3,ADAMANTINA,22-25 ANOS,FEMININO,296.0,288.0
4,ADAMANTINA,22-25 ANOS,MASCULINO,291.0,318.0


Garantir que as colunas S e N existem

In [21]:
if 'S' not in df_pivot.columns: df_pivot['S'] = 0
if 'N' not in df_pivot.columns: df_pivot['N'] = 0

Calcular KPI 1 (Taxa de Conversão)
Total de pessoas naquele perfil

In [22]:
df_pivot['Total_Habilitados'] = df_pivot['S'] + df_pivot['N']

In [23]:
df_pivot['KPI_Taxa_Conversao'] = df_pivot['S'] / df_pivot['Total_Habilitados']

In [24]:
df_pivot = df_pivot[df_pivot['Total_Habilitados'] > 0].copy()

In [25]:
print("Tabela Mestra criada. Visualizando amostra:")
df_pivot.head()

Tabela Mestra criada. Visualizando amostra:


,descricao_municipio,faixa_etaria,genero,N,S,Total_Habilitados,KPI_Taxa_Conversao
0,ADAMANTINA,101-120 ANOS,MASCULINO,17.0,0.0,17.0,0.000000
1,ADAMANTINA,18-21 ANOS,FEMININO,86.0,342.0,428.0,0.799065
2,ADAMANTINA,18-21 ANOS,MASCULINO,55.0,443.0,498.0,0.889558
3,ADAMANTINA,22-25 ANOS,FEMININO,296.0,288.0,584.0,0.493151
4,ADAMANTINA,22-25 ANOS,MASCULINO,291.0,318.0,609.0,0.522167


Preparar os Encoders (Transformar texto em número para se enquadrar a tipagem aceita pelo modelo de ML)

In [26]:
le_muni = LabelEncoder()
le_faixa = LabelEncoder()
le_genero = LabelEncoder()

In [27]:
df_pivot['code_muni'] = le_muni.fit_transform(df_pivot['descricao_municipio'])
df_pivot['code_faixa'] = le_faixa.fit_transform(df_pivot['faixa_etaria'])
df_pivot['code_genero'] = le_genero.fit_transform(df_pivot['genero'])

Definir X (Características) e y (O que ele vai aprender)

In [28]:
X = df_pivot[['code_muni', 'code_faixa', 'code_genero']]
y = df_pivot['KPI_Taxa_Conversao']

Treinar o Modelo (Random Forest)

In [29]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

RandomForestRegressor(random_state=42)

Gerar a Predição (KPI 3)

In [30]:
df_pivot['KPI_Score_Propensao'] = model.predict(X)

In [31]:
df_pivot.head()

,descricao_municipio,faixa_etaria,genero,N,S,Total_Habilitados,KPI_Taxa_Conversao,code_muni,code_faixa,code_genero,KPI_Score_Propensao
0,ADAMANTINA,101-120 ANOS,MASCULINO,17.0,0.0,17.0,0.000000,0,1,1,0.000000
1,ADAMANTINA,18-21 ANOS,FEMININO,86.0,342.0,428.0,0.799065,0,2,0,0.817397
2,ADAMANTINA,18-21 ANOS,MASCULINO,55.0,443.0,498.0,0.889558,0,2,1,0.873010
3,ADAMANTINA,22-25 ANOS,FEMININO,296.0,288.0,584.0,0.493151,0,3,0,0.522405
4,ADAMANTINA,22-25 ANOS,MASCULINO,291.0,318.0,609.0,0.522167,0,3,1,0.507330


Manipulando o DataFrame para exportar corretamente para o Power BI

In [32]:
df_final = df_pivot[[
    'descricao_municipio',
    'faixa_etaria',
    'genero',
    'S',                       # Volume Absoluto (KPI 2)
    'KPI_Taxa_Conversao',      # Taxa Real (KPI 1)
    'KPI_Score_Propensao'      # Score Preditivo (KPI 3)
]].rename(columns={
    'descricao_municipio': 'Cidade',
    'faixa_etaria': 'Faixa_Etaria',
    'genero': 'Genero',
    'S': 'KPI_Volume_Demanda'
})

In [33]:
# Arredondar para 4 casas decimais (fica melhor no Power BI)
df_final['KPI_Taxa_Conversao'] = df_final['KPI_Taxa_Conversao'].round(4)
df_final['KPI_Score_Propensao'] = df_final['KPI_Score_Propensao'].round(4)

# Salvar e Baixar
nome_saida = 'baseBICNH_dashboard_2025_fase2.csv'
df_final.to_csv(nome_saida, index=False)